## Import and Data Path Assignment

To begin our analysis, we first need to import the necessary libraries and assign the path to our data files. This step ensures that we have access to the tools and resources required for the subsequent tasks.

Let's get started by executing the following code:

In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler
%load_ext memory_profiler

import matplotlib.pyplot as plt
import skimage.io
import numpy as np
import tifffile

import nima_io.read as ir

from pathlib import Path

tdata = Path("../../tests/data/")

lif = tdata / "2015Aug28_TransHXB2_50min+DMSO.lif"
img_tile = tdata / "t4_1.tif"  # C=3 T=4 S=15
img_void_tile = tdata / "tile6_1.tif"  # C=4 T=3 S=14 scattered
imgsingle = tdata / "exp2_2.tif"  # C=2 T=81

mcts = tdata / "multi-channel-time-series.ome.tif"  # C=3 T=7
bigtiff = tdata / "LC26GFP_1.tf8"  # bigtiff

slif = str(lif)
simg_tile = str(img_tile)
simg_void_tile = str(img_void_tile)
simgsingle = str(imgsingle)
smcts = str(mcts)
sbigtiff = str(bigtiff)

So, we have few options to open microscopy datafiles:

- skimage.io.imread
- skimage.io.imread_collection
- tifffile.TiffFile
- tifffile.TiffSequence
- bioformats.ImageReader

Imagej hyperstack are organized as **TZCYXS**.

Holoview can also be used. Check availability of reading from disk a la memmap.

Bioformats claims the following standard: separate tiff for each channel and for each time point.
Thus a ome.tif would contain a single plane or a zstack. 

What about tiles? Bioformats has some 6D, 7D and 8D working aroud 5D. But how is exactly defined 5D?

## Skimage and Tifffile

In [ ]:
t1 = skimage.io.imread(img_tile, plugin='tifffile')
t2 = skimage.io.imread(img_void_tile, plugin='tifffile')
t1.shape, t2.shape

In [ ]:
tf1 = tifffile.imread(img_tile)
tf2 = tifffile.imread(img_void_tile)
tf1.shape, tf2.shape

Only for tiff data files provides:

- sequence
- OME metadata
- memmap
- zarr

In [ ]:
fp1glob = '/home/dati/liaisan/data/260816/FLAT/cyan_em10/cyan_em10_p*.tif'
tifs = tifffile.TiffSequence(fp1glob)
d = tifs.asarray()
print(d.shape) 
print(tifs.shape)

In [ ]:
with tifffile.TiffFile(img_tile) as tif:
    tag = tif.pages[0].tags['ImageDescription']
    
tag.value

## nima_io.read

In [ ]:
ir.ensure_vm()

In [ ]:
%%timeit
r = ir.read(simg_void_tile)

In [ ]:
r

In [ ]:
%%timeit
#md2, wr2, mdd2 = ir.read2(simg_void_tile,True)
r2 = ir.read2(simg_void_tile)

In [ ]:
r2[0]

In [ ]:
%%timeit
rinf = ir.read_inf(img_void_tile)

In [ ]:
rinf = ir.read_inf(img_void_tile)
rinf

In [ ]:
wr2 == wr

In [ ]:
import json

with open('md2.json', 'w') as f:
       json.dump(md2, f, indent=2)

with open('md2.json', 'r') as f:
       loaded_dict = json.load(f)

In [ ]:
np.save('md2.npy', md2)

In [ ]:
read_dictionary = np.load('md2.npy', allow_pickle=True).item()

In [ ]:
ir.get_md_dict(md_xml)

In [ ]:
mdJ, wrJ = ir.read_jpype(simg_void_tile)

In [ ]:
sorted_dict = {k: read_dictionary[k] for k in sorted(read_dictionary.keys())}
sorted_dictJ = {k: wrJ[2][k] for k in sorted(wrJ[2].keys())}

In [ ]:
sorted(wrJ[2].keys()) == sorted(read_dictionary.keys())

In [ ]:
read_dictionary.keys() - wrJ[2].keys()

In [ ]:
read_dictionary["Format"]

In [ ]:
read_dictionary.pop("Format")

In [ ]:
len(read_dictionary.keys() == 

In [ ]:
sorted_dict = {k: read_dictionary[k] for k in sorted(read_dictionary.keys())}
for k in sorted_dict:
    if not sorted_dict[k] == sorted_dictJ[k]:
        print(k)

In [ ]:
print(read_dictionary["ChannelLightSourceSettingsAttenuation"], wrJ[2]["ChannelLightSourceSettingsAttenuation"])
print(read_dictionary["PixelsBigEndian"], wrJ[2]["PixelsBigEndian"])
print(read_dictionary["PixelsInterleaved"], wrJ[2]["PixelsInterleaved"])

In [ ]:
%%timeit
rbf = ir.read_bf(simg_void_tile)

In [ ]:
rbf

In [ ]:
%%timeit
rjb = ir.read_jb(simg_void_tile)

In [ ]:
rjb

In [ ]:
%timeit rrw = ir.read_wrap(simg_void_tile)

In [ ]:
rrw

### PIMS

In [ ]:
%timeit ir.read_pims(img_void_tile)

In [ ]:
rpims = ir.read_pims(img_void_tile)
rpims

In [ ]:
%timeit ir.read_jpype(simg_void_tile)
%memit ir.read_jpype(simg_void_tile)

In [ ]:
rjpype = ir.read_jpype(simg_void_tile)
rjpype

In [ ]:
md1, md2

In [ ]:
f = ir.stitch(md2, wr2, c=2, t=2)
skimage.io.imshow(f)

In [ ]:
import javabridge
import bioformats
javabridge.start_vm(class_path=bioformats.JARS)

In [ ]:
bioformats.ImageReader(str(img_tile))

In [ ]:
import jpype, scyjava
scyjava.start_jvm()
def hello():
    JOptionPane = scyjava.jimport('javax.swing.JOptionPane')
    JOptionPane.showMessageDialog(None, "Hello world")
jpype.setupGuiEnvironment(hello)

#bioformats_jar
import jpype
import scyjava

scyjava.config.endpoints.append('ome:formats-gpl:6.7.0')
scyjava.start_jvm()
loci = jpype.JPackage("loci")
loci.common.DebugTools.setRootLevel("ERROR")

## performance
### showinf

In [ ]:
s1, s2, s3 = str(lif), str(imgsingle), str(img_tile)
%timeit ir.read_inf(s1), ir.read_inf(s2), ir.read_inf(s3)
#%memit ir.read_inf(lif), ir.read_inf(img), ir.read_inf(img_tile)

### bioformats (std)

In [ ]:
%timeit ir.read_bf(s1), ir.read_bf(s2), ir.read_bf(s3)
#%memit ir.read_bf(lif), ir.read_bf(img), ir.read_bf(img_tile)

### javabridge

In [ ]:
# OME-TIFF only
%timeit ir.read_jb(img), ir.read_jb(img_tile)
%memit ir.read_jb(img), ir.read_jb(img_tile)

### read

In [ ]:
%timeit ir.read(lif), ir.read(img), ir.read(img_tile)
%memit ir.read(lif), ir.read(img), ir.read(img_tile)

## ImageIO

Started comparing the available options to read microscopy file.

- mosaic

In [ ]:
from imageio.v3 import imread

%timeit imread(img_void_tile, index=13)
%memit imread(img_void_tile, index=13)
i = imread(img_void_tile, index=13)
i.shape

In [ ]:
i.nbytes, 512**2*3*4*2

It can read tif (tf8) files. Series might be passed using `index` (you need to know in advance).

## AICSImageIO

In [ ]:
from aicsimageio import AICSImage

i = AICSImage(img_void_tile)
#i = AICSImage(img_void_tile, reconstruct_mosaic=True)
#i_lif = AICSImage(lif)

In [ ]:
i.ome_metadata

In [ ]:
i_lif.scenes

In [ ]:
i.get_dask_stack()

Mosaic stitch is not supported on tif files; so I will use my function relying on the PositionXYZ metadata.

## dask_image

In [ ]:
from dask_image.imread import imread

i = imread(img_void_tile)

In [ ]:
i

In [ ]:
imread(lif)

Somehow it uses bioformats and can handle lif. No mosaic, no metadata though.

**Pycroscopy** https://pypi.org/project/pycroscopy/ is not reading lif nor ome-tif at the moment.

**large-image[all]** failed to intall.

**pyimagej** need conda?

## bioio-bioformats

In [ ]:
import bioio_ome_tiled_tiff

In [ ]:
bioio_ome_tiled_tiff.Reader(str(img_tile))

In [ ]:
import bioio_bioformats

im = bioio_bioformats.Reader(img_void_tile)
bioio_bioformats.ReaderMetadata(img_void_tile)

In [ ]:
im.xarray_dask_data

In [ ]:
i = bioio_bioformats.Reader(img_tile)
i.data.shape

In [ ]:
i.xarray_dask_data.attrs['processed']

In [ ]:
import nima_io.read as ir

In [ ]:
ir.bioformats.get_omexml_metadata(img_tile)

In [ ]:
unp = i.xarray_dask_data.attrs['unprocessed']
ir.get_md_dict?

In [ ]:
i.ome_metadata

In [ ]:
stk = i.get_dask_stack()

In [ ]:
stk.A

## bfio

In [ ]:
import bfio

bfio.BioReader(img_void_tile)

In [ ]:
rdr = bfio.BioReader(img_void_tile)
%timeit i = rdr.read()
i = rdr.read()
i.shape

In [ ]:
rdr.metadata

In [ ]:
rdr.ps_x

In [ ]:
rdr.close()

## PIMS

In [ ]:
import pims

%timeit fs = pims.Bioformats(img_void_tile)
fs = pims.Bioformats(img_void_tile)
fs.sizes

## PyOMETiff

In [ ]:
import pyometiff

%timeit rdr = pyometiff.OMETIFFReader(fpath=img_void_tile)
rdr = pyometiff.OMETIFFReader(fpath=img_void_tile)

In [ ]:
%timeit r = rdr.read()
res = rdr.read()

In [ ]:
res[2]

In [ ]:
pyometiff.OMETIFFReader._get_metadata_template()

## Final Note

I will keep 

0. Read
1. stitch
2. md_grouping


- pycromanager
- microscope
- python-microscopy

In [ ]:
import imgread

In [ ]:
from imgread import read

In [ ]:
read.read_pims(img_void_tile)

In [ ]:
read.read_jb(img_void_tile)